Reference: [Build a Retrieval Augmented Generation (RAG) App](https://python.langchain.com/v0.2/docs/tutorials/rag/)

In [5]:
import dotenv
from langchain_openai import ChatOpenAI

dotenv.load_dotenv() # invoke langsmith

llm = ChatOpenAI(model="gpt-3.5-turbo")

## Build vector database

In [1]:
import os

from langchain import hub
from langchain_community.document_loaders import UnstructuredMarkdownLoader
from langchain_community.vectorstores import Qdrant
from langchain_openai import OpenAIEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter

embedding_cache_path = "../embedding_cache/"
db_collection_names = ["CNS16190_md", "CNS16190_md_only_provision"]
document_root_path = "../docs"
documents = ["CNS16190-zh_TW.md", "CNS16190-zh_TW_only_provision.md"]

collection = db_collection_names[0]
if os.path.isdir(os.path.join(embedding_cache_path, "collection", collection)):
    # database already exists, load it
    print("Loading existing database")
    vectorstore = Qdrant.from_existing_collection(
        embedding=OpenAIEmbeddings(),
        path=embedding_cache_path,
        collection_name=collection,
    )
else:
    # database does not exist, create it
    print("Creating new database")
    loader = UnstructuredMarkdownLoader(
        os.path.join(document_root_path, documents[0]), mode="elements"
    )
    doc = loader.load()

    text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
    splits = text_splitter.split_documents(doc)

    vectorstore = Qdrant.from_documents(
        splits,
        embedding=OpenAIEmbeddings(),
        path=embedding_cache_path,
        collection_name=collection,
    )

retriever = vectorstore.as_retriever()

Loading existing database


In [2]:

prompt = hub.pull(
    "rlm/rag-prompt"
)  # https://smith.langchain.com/hub/rlm/rag-prompt?organizationId=07050030-99d2-5925-9ec6-8a8fb67164d3

prompt.messages[0].prompt.template="你是一個資安專家，使用以下檢索到的背景資料回答問題，如果不知道答案就說不知道。\n背景資料：{context} \n問題：{question} \n答案："
prompt


ChatPromptTemplate(input_variables=['context', 'question'], metadata={'lc_hub_owner': 'rlm', 'lc_hub_repo': 'rag-prompt', 'lc_hub_commit_hash': '50442af133e61576e74536c6556cefe1fac147cad032f4377b60c436e6cdcb6e'}, messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context', 'question'], template='你是一個資安專家，使用以下檢索到的背景資料回答問題，如果不知道答案就說不知道。\n背景資料：{context} \n問題：{question} \n答案：'))])

## Build rag chain

In [3]:
import pandas as pd


# concat all docs as a string
def format_docs(docs):
    retrieval_docs = "\n\n".join(doc.page_content for doc in docs)
    df = pd.read_csv("../docs/test_data_from_GoogleSheet.csv")
    test_data = "\n\n測試資料範例：\n\n"
    for idx in range(3):
        row = df.iloc[idx]
        test_data += f"控制措施{row['provision']}，detail:{row['detail']}，question:{row['Test cenario']}，answer:{row['answer']}\n\n"

    result = retrieval_docs + test_data
    print(result)
    return result


In [6]:
from langchain_core.runnables import RunnablePassthrough

rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | llm
)
# This is equivalent to following without using format_docs():
# question_answer_chain = create_stuff_documents_chain(llm, prompt)
# rag_chain = create_retrieval_chain(retriever, question_answer_chain)

# the path of the rag_chain:
# 1. get "question" from rag_chain.invoke()
# 2. search the most relevant context from the retriever
# 3. concat the docs from retriever in format_docs() as a string
# 4. format the string as a prompt
# 5. query the prompt to the llm

## Ask llm

In [7]:
df = pd.read_csv("../docs/test_data_from_GoogleSheet.csv")

df

,provision,status,support,detail,Test cenario,answer,reason,result
0,5.1-1,MC,NaN,如果使用密碼並且處於出廠預設設置以外的任何狀態，則所有消費者物聯網設備密碼應為每個設備獨有的...,問題：當使用密碼且處於出廠預設值（例如「admin」）以外的任何狀態時，所有消費者物聯網裝置...,使用者密碼透過wizard由使用者設定且與MAC Address/SN無相關性，且無法反算或...,NaN,NaN
1,5.1-3,M,NaN,用以對裝置鑑別使用者之鑑別機制，應使用對技術、風險及使用的性質適切之最佳實務密碼學。,與裝置所使用的身份驗證方法無關，身份驗證資料在使用者和裝置之間傳輸時應進行加密。注意：傳輸通...,本機驗證透過 HMAC-SHA256 質詢使用使用者設定的密碼完成。 透過 HTTPs TL...,NaN,NaN
2,5.2-1,M,NaN,製造商應公開漏洞公布政策。本政策至少應包括：報告問題的聯繫資訊;以及關於以下方面的時程表的資...,漏洞揭露政策明確規定了安全研究人員和其他人報告問題的流程。 此類政策可以根據需要進行更新，以...,通過D-Link 網站（https://support.dlink.com/ReportVu...,NaN,NaN
3,5.3-13,M,NaN,製造商應以對使用者易於理解、透明的方式公開定義的支援期限。,消費者在購買設備時，希望能明確說明該期限的軟體更新支援。問題：1. 客戶如何獲知定義的支援期...,產品保固期間都根據當地國家法規要求訂定之，產品生命維護週期到時，會提前60天前公告在產品網頁上。,NaN,NaN
4,5.3-16,M,NaN,消費者物聯網設備的型號名稱應通過設備上的標籤或通過實體介面清晰可識別。,問題：1. 使用者如何驗證設備型號名稱（透過設備上的標籤或透過實體介面）？,此產品的型號標籤上有相關的生產編號資訊。,NaN,NaN
5,5.4-1,M,NaN,持久存儲中的敏感安全參數應由設備安全的存儲。,安全、可信任的儲存機制可用於保護敏感的安全參數。注意：此規定適用於持久性儲存（在裝置電源關閉...,所有關鍵機敏資訊都以AES 128 加密演算法處理存儲在機器的flash中。,NaN,NaN
6,5.4-3,M,NaN,關鍵安全參數不得硬編碼於使用設備軟體原始碼中。,設備和應用程式的逆向工程可以輕鬆發現憑證，例如軟體中硬編碼的使用者名稱和密碼。注意：此類憑證...,D-Link 產品都會經過縝密的源碼安全檢查掃描，以確保沒有相關的 Hard-coded c...,NaN,NaN
7,5.4-4,M,NaN,用於軟體更新的完整性和真實性檢查，以及保護設備軟體中相關服務的通訊的任何關鍵安全參數應為每個...,為設備配置獨特的關鍵安全參數有助於保護軟體更新的完整性和真實性以及設備與相關服務的通訊。如果...,D-Link Cloud 管理通訊使用每個設備唯一的設備證書，在設定過程中進行配置。\r\n...,NaN,NaN
8,5.5-1,M,NaN,消費者物聯網設備應使用最佳實踐加密技術進行安全通訊。,安全控制的適當性和最佳實踐加密的使用取決於許多因素，包括使用情境。注意：由於安全性不斷發展，...,D-Link 網站的所有通訊，包含FW檔案的更新作業都是在全程TLS V1.2以上的安全加密...,NaN,NaN
9,5.5-5,M,NaN,設備功能只有在認證存取之後允許透過網路介面進行配置安全相關修改。例外情況是設備所依賴的網路服...,注意：例外的協定包括 ARP、DHCP、DNS、ICMP 和 NTP。範例：與安全性相關的變...,D-Link 網站的所有通訊，包含FW檔案的更新作業都是在全程TLS V1.2以上的安全加密...,NaN,NaN


### One row testing

In [9]:
row = df.iloc[7]
query = f"控制措施{row['provision']}，detail:{row['detail']}，answer:{row['answer']}"
print(
    f"控制措施 {row['provision']}\n\n【detail】\n{row['detail']}\n\n【question】\n{row['Test cenario']}\n\n【answer】\n{row['answer']}"
)

print(f"\n【question】\n{row['Test cenario']}")


控制措施 5.4-4

【detail】
用於軟體更新的完整性和真實性檢查，以及保護設備軟體中相關服務的通訊的任何關鍵安全參數應為每個設備獨有，並且應具有降低對設備類別進行自動攻擊風險的機制。

【question】
為設備配置獨特的關鍵安全參數有助於保護軟體更新的完整性和真實性以及設備與相關服務的通訊。如果使用全球關鍵安全參數，其洩漏可能會導致對其他物聯網設備的大規模攻擊，例如建立殭屍網路。範例：在同一裝置類別的每個裝置上部署不同的對稱金鑰，用於產生和驗證軟體更新的訊息認證碼。範例：設備使用製造商的公鑰來驗證軟體更新。 這不是關鍵的安全參數，並且不需要每個設備都是唯一的。問題：對於設備中用於以下用途的任何關鍵安全參數：- 軟體更新的完整性和真實性檢查；- 保護與設備軟體中相關服務的通訊：1. 這些關鍵安全參數是：A) 每個設備都是唯一的？B) 採用可降低針對各類設備的自動攻擊風險的機制進行生產？

【answer】
D-Link Cloud 管理通訊使用每個設備唯一的設備證書，在設定過程中進行配置。
管理和設定通道始終透過 TLS v1.2 或更新的連線進行保護，chiper 套件符合 FIPS 14--3。
伺服器證書還可以透過設備的嵌入式證書頒發機構文件進行驗證。

【question】
為設備配置獨特的關鍵安全參數有助於保護軟體更新的完整性和真實性以及設備與相關服務的通訊。如果使用全球關鍵安全參數，其洩漏可能會導致對其他物聯網設備的大規模攻擊，例如建立殭屍網路。範例：在同一裝置類別的每個裝置上部署不同的對稱金鑰，用於產生和驗證軟體更新的訊息認證碼。範例：設備使用製造商的公鑰來驗證軟體更新。 這不是關鍵的安全參數，並且不需要每個設備都是唯一的。問題：對於設備中用於以下用途的任何關鍵安全參數：- 軟體更新的完整性和真實性檢查；- 保護與設備軟體中相關服務的通訊：1. 這些關鍵安全參數是：A) 每個設備都是唯一的？B) 採用可降低針對各類設備的自動攻擊風險的機制進行生產？


In [10]:
rag_chain.invoke(f"{query}\n\n針對控制措施、detail和answer反推出疑問question")

控制措施5.4-4：用於軟體更新之完整性及真確性的核對，以及用於保護與裝置軟體中相關聯服務之通訊的任何關鍵安全參數，應為每裝置唯一，並應以降低對各類別裝置自動化攻擊的機制產生。

控制措施5.4-4：用於軟體更新之完整性及真確性的核對，以及用於保護與裝置軟體中相關聯服務之通訊的任何關鍵安全參數，應為每裝置唯一，並應以降低對各類別裝置自動化攻擊的機制產生。

控制措施5.4-4：用於軟體更新之完整性及真確性的核對，以及用於保護與裝置軟體中相關聯服務之通訊的任何關鍵安全參數，應為每裝置唯一，並應以降低對各類別裝置自動化攻擊的機制產生。

控制措施5.4-4：用於軟體更新之完整性及真確性的核對，以及用於保護與裝置軟體中相關聯服務之通訊的任何關鍵安全參數，應為每裝置唯一，並應以降低對各類別裝置自動化攻擊的機制產生。

測試資料範例：

控制措施5.1-1 ，detail:如果使用密碼並且處於出廠預設設置以外的任何狀態，則所有消費者物聯網設備密碼應為每個設備獨有的或由用戶定義。 ，question:問題：當使用密碼且處於出廠預設值（例如「admin」）以外的任何狀態時，所有消費者物聯網裝置密碼應符合下列其中一項：1. 每個設備都是唯一的。2. 由使用者定義。請詳細說明密碼是如何建立的。 ，answer:使用者密碼透過wizard由使用者設定且與MAC Address/SN無相關性，且無法反算或暴力計算。

控制措施5.1-3，detail:用以對裝置鑑別使用者之鑑別機制，應使用對技術、風險及使用的性質適切之最佳實務密碼學。，question:與裝置所使用的身份驗證方法無關，身份驗證資料在使用者和裝置之間傳輸時應進行加密。注意：傳輸通常可以在行動應用程式和裝置之間進行。注意：密碼需要始終受到保護，加密是保護密碼的標準方法。 如果密碼未加密，則它們可以簡單地讀取（即明文）並且很容易洩漏。（範例：傳輸過程中的密碼保護通常可以透過 TLS/SSL 隧道連線或 VPN 進行）。問題：1. 當使用者向裝置進行身份驗證時，身份驗證資料在傳輸過程中如何進行加密保護？ ，answer:本機驗證透過 HMAC-SHA256 質詢使用使用者設定的密碼完成。 透過 HTTPs TLS v1.2 或更新的連線保護通道，chiper 套件符合 FIPS 14--3
雲端管理通訊也透過 TLS v1.2 

AIMessage(content='問題：控制措施5.4-4中提到的軟體更新完整性和真實性檢查以及保護通訊的安全參數，為什麼需要為每個設備唯一？這樣做有哪些好處？', response_metadata={'token_usage': {'completion_tokens': 94, 'prompt_tokens': 2212, 'total_tokens': 2306}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-763c6d5d-e15f-46ac-8b1a-06c6b8e43e62-0', usage_metadata={'input_tokens': 2212, 'output_tokens': 94, 'total_tokens': 2306})

### Batch test

In [16]:
queries = []
for row in range(len(df) - 3):
    row = df.iloc[row + 3]
    condition = (
        f"控制措施{row['provision']}，detail:{row['detail']}，answer:{row['answer']}"
    )
    query = f"{condition}\n\n針對控制措施、detail和answer反推出疑問question"
    queries.append(query)
queries


['控制措施5.3-13，detail:製造商應以對使用者易於理解、透明的方式公開定義的支援期限。，answer:產品保固期間都根據當地國家法規要求訂定之，產品生命維護週期到時，會提前60天前公告在產品網頁上。\n\n針對控制措施、detail和answer反推出疑問question',
 '控制措施5.3-16，detail:消費者物聯網設備的型號名稱應通過設備上的標籤或通過實體介面清晰可識別。，answer:此產品的型號標籤上有相關的生產編號資訊。\n\n針對控制措施、detail和answer反推出疑問question',
 '控制措施5.4-1，detail:持久存儲中的敏感安全參數應由設備安全的存儲。，answer:所有關鍵機敏資訊都以AES 128 加密演算法處理存儲在機器的flash中。\n\n針對控制措施、detail和answer反推出疑問question',
 '控制措施5.4-3，detail:關鍵安全參數不得硬編碼於使用設備軟體原始碼中。，answer:D-Link 產品都會經過縝密的源碼安全檢查掃描，以確保沒有相關的 Hard-coded critical security parameters 在程式中\n\n針對控制措施、detail和answer反推出疑問question',
 '控制措施5.4-4，detail:用於軟體更新的完整性和真實性檢查，以及保護設備軟體中相關服務的通訊的任何關鍵安全參數應為每個設備獨有，並且應具有降低對設備類別進行自動攻擊風險的機制。，answer:D-Link Cloud 管理通訊使用每個設備唯一的設備證書，在設定過程中進行配置。\r\n管理和設定通道始終透過 TLS v1.2 或更新的連線進行保護，chiper 套件符合 FIPS 14--3。\r\n伺服器證書還可以透過設備的嵌入式證書頒發機構文件進行驗證。\n\n針對控制措施、detail和answer反推出疑問question',
 '控制措施5.5-1，detail:消費者物聯網設備應使用最佳實踐加密技術進行安全通訊。，answer:D-Link 網站的所有通訊，包含FW檔案的更新作業都是在全程TLS V1.2以上的安全加密方式進行，並採用最高等級的Cipher 符合FIPS 14--3 的安全強度模組進行，並且會隨時注意最先安全通報，檢查是否現

In [17]:
results = rag_chain.batch(queries)

控制措施5.3-16：消費者IoT裝置之型號名稱應藉由於裝置上加標籤，或經由實體介面可清晰辨識。

控制措施5.3-16：消費者IoT裝置之型號名稱應藉由於裝置上加標籤，或經由實體介面可清晰辨識。

控制措施5.3-16：消費者IoT裝置之型號名稱應藉由於裝置上加標籤，或經由實體介面可清晰辨識。

控制措施5.3-16：消費者IoT裝置之型號名稱應藉由於裝置上加標籤，或經由實體介面可清晰辨識。

測試資料範例：

控制措施5.1-1 ，detail:如果使用密碼並且處於出廠預設設置以外的任何狀態，則所有消費者物聯網設備密碼應為每個設備獨有的或由用戶定義。 ，question:問題：當使用密碼且處於出廠預設值（例如「admin」）以外的任何狀態時，所有消費者物聯網裝置密碼應符合下列其中一項：1. 每個設備都是唯一的。2. 由使用者定義。請詳細說明密碼是如何建立的。 ，answer:使用者密碼透過wizard由使用者設定且與MAC Address/SN無相關性，且無法反算或暴力計算。

控制措施5.1-3，detail:用以對裝置鑑別使用者之鑑別機制，應使用對技術、風險及使用的性質適切之最佳實務密碼學。，question:與裝置所使用的身份驗證方法無關，身份驗證資料在使用者和裝置之間傳輸時應進行加密。注意：傳輸通常可以在行動應用程式和裝置之間進行。注意：密碼需要始終受到保護，加密是保護密碼的標準方法。 如果密碼未加密，則它們可以簡單地讀取（即明文）並且很容易洩漏。（範例：傳輸過程中的密碼保護通常可以透過 TLS/SSL 隧道連線或 VPN 進行）。問題：1. 當使用者向裝置進行身份驗證時，身份驗證資料在傳輸過程中如何進行加密保護？ ，answer:本機驗證透過 HMAC-SHA256 質詢使用使用者設定的密碼完成。 透過 HTTPs TLS v1.2 或更新的連線保護通道，chiper 套件符合 FIPS 14--3
雲端管理通訊也透過 TLS v1.2 或更新的連線進行保護，chiper 套件符合 FIPS 14--3。 設備與雲端之間透過設備證書進行身份驗證。

控制措施5.2-1，detail:製造商應公開漏洞公布政策。本政策至少應包括：報告問題的聯繫資訊;以及關於以下方面的時程表的資訊：1）收到初始確認;以及 2）狀態更新，直到報告的問題得到解決。，questi

In [18]:
results

[AIMessage(content='製造商應以對使用者易於理解、透明的方式公開定義的支援期限。問題：製造商如何確定並公開產品的支援期限？', response_metadata={'token_usage': {'completion_tokens': 62, 'prompt_tokens': 1699, 'total_tokens': 1761}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-617ef220-a54c-406e-aa37-2fae6ef3cda5-0', usage_metadata={'input_tokens': 1699, 'output_tokens': 62, 'total_tokens': 1761}),
 AIMessage(content='消費者物聯網設備的型號名稱應如何清晰可識別？', response_metadata={'token_usage': {'completion_tokens': 36, 'prompt_tokens': 1769, 'total_tokens': 1805}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-c33923ad-ee64-414b-b1fd-9c9ce90c63a5-0', usage_metadata={'input_tokens': 1769, 'output_tokens': 36, 'total_tokens': 1805}),
 AIMessage(content='問題：持久性儲存體中的敏感性安全參數應由裝置安全儲存時，這些敏感性安全參數是如何被加密和存儲的？', response_metadata={'token_usage': {'completion_tokens': 69, 'prompt_tokens': 1675, 'total_tokens': 1744}, 'model_name':

In [25]:
question_list = []
for result in results:
    question_list.append(result.content)


data = {
    "provision": df["provision"].iloc[3:].tolist(),
    "detail": df["detail"].iloc[3:].tolist(),
    "qustion": question_list,
    "answer": df["answer"].iloc[3:].tolist(),
}

result_df = pd.DataFrame(data)
result_df

,provision,detail,qustion,answer
0,5.3-13,製造商應以對使用者易於理解、透明的方式公開定義的支援期限。,製造商應以對使用者易於理解、透明的方式公開定義的支援期限。問題：製造商如何確定並公開產品的支...,產品保固期間都根據當地國家法規要求訂定之，產品生命維護週期到時，會提前60天前公告在產品網頁上。
1,5.3-16,消費者物聯網設備的型號名稱應通過設備上的標籤或通過實體介面清晰可識別。,消費者物聯網設備的型號名稱應如何清晰可識別？,此產品的型號標籤上有相關的生產編號資訊。
2,5.4-1,持久存儲中的敏感安全參數應由設備安全的存儲。,問題：持久性儲存體中的敏感性安全參數應由裝置安全儲存時，這些敏感性安全參數是如何被加密和存儲的？,所有關鍵機敏資訊都以AES 128 加密演算法處理存儲在機器的flash中。
3,5.4-3,關鍵安全參數不得硬編碼於使用設備軟體原始碼中。,問題：控制措施5.4-3，硬編碼關鍵安全參數在裝置軟體原始碼中的使用情況如何？,D-Link 產品都會經過縝密的源碼安全檢查掃描，以確保沒有相關的 Hard-coded c...
4,5.4-4,用於軟體更新的完整性和真實性檢查，以及保護設備軟體中相關服務的通訊的任何關鍵安全參數應為每個...,不知道。,D-Link Cloud 管理通訊使用每個設備唯一的設備證書，在設定過程中進行配置。\r\n...
5,5.5-1,消費者物聯網設備應使用最佳實踐加密技術進行安全通訊。,對不起，我無法回答你的問題。,D-Link 網站的所有通訊，包含FW檔案的更新作業都是在全程TLS V1.2以上的安全加密...
6,5.5-5,設備功能只有在認證存取之後允許透過網路介面進行配置安全相關修改。例外情況是設備所依賴的網路服...,不知道。,D-Link 網站的所有通訊，包含FW檔案的更新作業都是在全程TLS V1.2以上的安全加密...
7,5.5-7,消費者物聯網設備應保護透過遠端存取網路介面的關鍵安全參數的機密性。,控制措施5.5-7，question:消費者IoT裝置經由可遠端接取之網路介面進行通訊時，關...,D-Link 網站的所有通訊，包含FW檔案的更新作業都是在全程TLS V1.2以上的安全加密...
8,5.5-8,製造商應遵循與設備相關的關鍵安全參數的安全管理流程。,問題：製造商應遵循與設備相關的關鍵安全參數的安全管理流程。請問製造商如何確保安全參數的生成、...,按照D-Link的秘鑰管理流程生成、存儲和傳輸安全參數。 安全參數隨機生成后，寫入到設備中，...
9,5.6-1,應禁用所有未使用的網路和邏輯介面。,問題：為什麼應該停用所有未使用的網路和邏輯介面？,有一個 LAN 的乙太網路、2.4G 和 5G Wi-Fi 接口，需要在設備使用過程中運行，...


In [26]:
question_data = {
    "provision": df["provision"].iloc[3:].tolist(),
    "created qustion": question_list,
    "provided qustion": df["Test cenario"].iloc[3:].tolist(),
    "answer": df["answer"].iloc[3:].tolist(),
}

question_df = pd.DataFrame(question_data)
question_df

,provision,created qustion,provided qustion,answer
0,5.3-13,製造商應以對使用者易於理解、透明的方式公開定義的支援期限。問題：製造商如何確定並公開產品的支...,消費者在購買設備時，希望能明確說明該期限的軟體更新支援。問題：1. 客戶如何獲知定義的支援期...,產品保固期間都根據當地國家法規要求訂定之，產品生命維護週期到時，會提前60天前公告在產品網頁上。
1,5.3-16,消費者物聯網設備的型號名稱應如何清晰可識別？,問題：1. 使用者如何驗證設備型號名稱（透過設備上的標籤或透過實體介面）？,此產品的型號標籤上有相關的生產編號資訊。
2,5.4-1,問題：持久性儲存體中的敏感性安全參數應由裝置安全儲存時，這些敏感性安全參數是如何被加密和存儲的？,安全、可信任的儲存機制可用於保護敏感的安全參數。注意：此規定適用於持久性儲存（在裝置電源關閉...,所有關鍵機敏資訊都以AES 128 加密演算法處理存儲在機器的flash中。
3,5.4-3,問題：控制措施5.4-3，硬編碼關鍵安全參數在裝置軟體原始碼中的使用情況如何？,設備和應用程式的逆向工程可以輕鬆發現憑證，例如軟體中硬編碼的使用者名稱和密碼。注意：此類憑證...,D-Link 產品都會經過縝密的源碼安全檢查掃描，以確保沒有相關的 Hard-coded c...
4,5.4-4,不知道。,為設備配置獨特的關鍵安全參數有助於保護軟體更新的完整性和真實性以及設備與相關服務的通訊。如果...,D-Link Cloud 管理通訊使用每個設備唯一的設備證書，在設定過程中進行配置。\r\n...
5,5.5-1,對不起，我無法回答你的問題。,安全控制的適當性和最佳實踐加密的使用取決於許多因素，包括使用情境。注意：由於安全性不斷發展，...,D-Link 網站的所有通訊，包含FW檔案的更新作業都是在全程TLS V1.2以上的安全加密...
6,5.5-5,不知道。,注意：例外的協定包括 ARP、DHCP、DNS、ICMP 和 NTP。範例：與安全性相關的變...,D-Link 網站的所有通訊，包含FW檔案的更新作業都是在全程TLS V1.2以上的安全加密...
7,5.5-7,控制措施5.5-7，question:消費者IoT裝置經由可遠端接取之網路介面進行通訊時，關...,可以使用加密的通訊通道或有效負載加密來實現機密性保護。注意：加密保護通常是使用至少與傳輸的密...,D-Link 網站的所有通訊，包含FW檔案的更新作業都是在全程TLS V1.2以上的安全加密...
8,5.5-8,問題：製造商應遵循與設備相關的關鍵安全參數的安全管理流程。請問製造商如何確保安全參數的生成、...,注意：對關鍵安全參數使用開放的、同儕審查的標準（通常稱為「金鑰管理」）受到強烈鼓勵。問題：當...,按照D-Link的秘鑰管理流程生成、存儲和傳輸安全參數。 安全參數隨機生成后，寫入到設備中，...
9,5.6-1,問題：為什麼應該停用所有未使用的網路和邏輯介面？,範例：預設情況下，無法從 WAN 存取應從 LAN 存取的管理 UI。注意：邏輯介面 (LI...,有一個 LAN 的乙太網路、2.4G 和 5G Wi-Fi 接口，需要在設備使用過程中運行，...


In [28]:
question_df.to_csv("created_questions.csv", index=False, encoding="utf-8-sig")